In [1]:
from __future__ import print_function, division
import os
import torch
from torch import nn
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import json
from transformers import TransfoXLTokenizer, TransfoXLModel

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pickle

pickle_file = 'video_descriptions/generic_video_valD'
output_folder = '../per_video_file/testD/'
last_one = 12000        

read_file = open(pickle_file, "rb")
data_items = pickle.load(read_file)
data_items = data_items['desc_list'][last_one:(last_one+64)]
print(len(data_items))

total_length = len(data_items)
read_file.close()

64


In [3]:
tokenizer = TransfoXLTokenizer.from_pretrained('transfo-xl-wt103')
pretrained_model = TransfoXLModel.from_pretrained('transfo-xl-wt103')

object_max_len = 110


        
        
for iter in range(total_length):#(0,2):#
    
    
    
#     print(data_items[iter]['description'])
    
    keys = {}
    keys["source"] = data_items[iter]['source']
    keys["objects_color"] = data_items[iter]['objects_color']
    keys["objects_size"] = data_items[iter]['objects_size']
    keys["objects_shape"] = data_items[iter]['objects_shape']
    keys["objects_material"] = data_items[iter]['objects_material']
    keys["objects_motion"] = data_items[iter]['objects_motion']
    keys["description"] = data_items[iter]['description']
    keys["length"] = data_items[iter]['length']
    
    hidden_state, _ = pretrained_model(torch.tensor(tokenizer.encode(keys["description"], add_special_tokens=True, 
                add_space_before_punct_symbol=True)).unsqueeze(0))
        
    last_hidden = hidden_state[-1][-1]#.view(1,1,-1)
    val = (object_max_len - list(hidden_state.shape)[1])


            # https://pytorch.org/docs/stable/nn.html#constantpad2d
    hidden_state = nn.ConstantPad2d((0, 0, 0, val), 0)(hidden_state)
    
    keys['hidden_state'] = hidden_state
    keys['last_hidden'] = last_hidden
    
    desc_dict = {'desc_list': keys}
    # with open('generic_description.json', 'w') as json_file:
    #     json.dump(description_list, json_file, indent = 4, sort_keys=True)
   
    i = iter #+ last_one


    save_file_name = output_folder + str(i)

    with open(save_file_name, 'wb') as outfile:
        pickle.dump(desc_dict,outfile)        
        